In [49]:
from typing import Callable
from pathlib import Path
import itertools

cat = ''.join

def mapt(func, *args):
    return tuple(map(func, *args))

def ints(line):
    return mapt(int, line.split())

lines = str.splitlines


def Input(day, parser: Callable=str, sections: Callable=lines, is_test=False):
    test = 'test' if is_test else ''
    file_path = f"inputs/{day}{test}.txt"
    text = Path(file_path).read_text()
    return mapt(parser, sections(text))

## Day 1

Part 1

In [36]:
ids = Input(1, parser=ints)
ids[:5]

((37033, 48086),
 (80098, 34930),
 (88073, 69183),
 (54342, 63061),
 (98409, 87908))

In [ ]:
left, right = list(zip(*ids))
total_distance = sum(abs(l - r) for l, r in zip(sorted(left), sorted(right)))
total_distance

1223326

Part 2

In [35]:
from collections import Counter
counts = Counter(right)
similarity = sum(n * counts[n] for n in left)
similarity

21070419

## Day 2

Part 1

In [42]:
reports = Input(2, parser=ints)
reports[:3]

((16, 18, 20, 22, 23, 22), (9, 10, 12, 13, 13), (64, 67, 69, 70, 74))

In [45]:
def sliding_window(iterable, n):
    return zip(*(iterable[i:] for i in range(n)))

def is_increasing(report):
    return all(a < b for a, b in sliding_window(report, 2))

def is_decreasing(report):
    return all(a > b for a, b in sliding_window(report, 2))

def has_safe_changes(report):
    return all(1 <= abs(a - b) <= 3 for a, b in sliding_window(report, 2))

def is_safe(report):
    return ((is_increasing(report) or is_decreasing(report))
            and has_safe_changes(report))

In [46]:
sum(is_safe(report) for report in reports)

606

Part 2

In [ ]:
def drop_one(report):
    for i in range(len(report)):
        yield report[:i] + report[i+1:]

In [ ]:
sum(any(is_safe(removed)
        for removed in drop_one(report))
    for report in reports)

644

## Day 3

Part 1

In [86]:
prog = cat(Input(3))
prog[:50]

'mul(168,87)}*:mul(911,800)(%,)where()#&&$mul(734,1'

In [88]:
import re

muls = re.compile(r'mul\((\d+),(\d+)\)').findall

sum(int(a) * int(b) for a, b in muls(prog))


166630675

Part 2

In [90]:
runnable = re.sub(r"don't\(\).*?(do\(\)|$)", ' ', prog)
sum(int(a) * int(b) for a, b in muls(runnable))

93465710

## Day 4

Part 1

In [99]:
grid = Input(4, is_test=False)
N = len(grid) # grid is square
N

140

In [100]:
directions4 = East, South, West, North = ((1, 0), (0, 1),  (-1, 0), (0, -1))
diagonals   = SE,   NE,    SW,   NW    = ((1, 1), (1, -1), (-1, 1), (-1, -1))
directions8 = directions4 + diagonals

def add(p, q):
    "Add 2D points"
    return (p[0] + q[0], p[1] + q[1])

def in_range(p, N=N):
    x, y = p
    return (0 <= x < N) and (0 <= y < N)

def Line(p, direction, n=4):
    "Returns n points starting at p along direction"
    res = [p]
    for i in range(n - 1):
        res.append(add(res[-1], direction))
    return res

def candidates(p):
    "All length-4 Lines starting at p that lie inside grid"
    res = []
    for direction in directions8:
        candidate = Line(p, direction)
        if all(in_range(q) for q in candidate):
            res.append(candidate)
    return res

def is_xmas(line):
    return all(grid[y][x] == letter for (x, y), letter in zip(line, 'XMAS'))

In [101]:
len([line 
     for p in itertools.product(range(N), range(N))
     for line in candidates(p)
     if is_xmas(line)])

2685

Part 2

In [102]:
def is_x_mas(p):
    "True if p is the center of an X-MAS. Assumes that 1 <= p < N - 1."
    corners = tl, bl, tr, br = [
        grid[y][x]
        for x, y in [add(p, NW), add(p, SW), add(p, NE), add(p, SE)]
    ]
    center = grid[p[1]][p[0]]
    return (center == 'A'
            and ([tl, br] == ['M', 'S'] or [tl, br] == ['S', 'M'])
            and ([bl, tr] == ['M', 'S'] or [bl, tr] == ['S', 'M']))

In [103]:
sum(is_x_mas(p) for p in itertools.product(range(1, N - 1), range(1, N - 1)))

2048